## Question 6

## Multiclass Perceptron

It is a non-probabilistic discriminative kind of classification, which works based on the below algorithm and uses decision boundary line to separate the classes.

<img src = "multi.png"/>

## Implementing a Multi-Class Perceptron Model

## 6.1 Loading data sets

In [25]:
#importing libraries
library(reshape2)
library(ggplot2)

In [26]:
#reading the training data set and displaying a few lines of trained data
train_data <- read.csv("Task1D_train.csv")
head(train_data)

x1,x2,x3,x4,y
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
5.1,3.5,1.4,0.2,C1
4.7,3.2,1.3,0.2,C1
5.0,3.6,1.4,0.2,C1
4.6,3.4,1.4,0.3,C1
4.4,2.9,1.4,0.2,C1
5.4,3.7,1.5,0.2,C1


In [27]:
#reading the test data set and displaying a few lines of test data
test_data <- read.csv("Task1D_test.csv")
head(test_data)

x1,x2,x3,x4,y
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
4.9,3.0,1.4,0.2,C1
4.6,3.1,1.5,0.2,C1
5.4,3.9,1.7,0.4,C1
5.0,3.4,1.5,0.2,C1
4.9,3.1,1.5,0.1,C1
4.8,3.4,1.6,0.2,C1


In [28]:
#splitting the train data set into data and labels
train.data = train_data[,-5]
train.label = train_data[,5]

#splitting the test data set into data and labels
test.data = test_data[,-5]
test.label = test_data[,5]

#scaling the train and test data sets
train.data = scale(train.data)
test.data = scale(test.data)

In [29]:
#obtaining the length of train and test data
train.len <- nrow(train.data)
test.len <- nrow(test.data)

## Question 6.2:- Implementing a multiclass perceptron model based on the above algorithm

In [30]:
multiclassfier_perceptron <- function(train.data, train.label, eta, epsilon , tau.max ) {
    
    #adding a new column to the train data which represents the coefficients namely w0
    Phi <- as.matrix(cbind(1, train.data)) 
    #convergence of train label
    T <- train.label 
    
    #creating three empty class matrices as our test label consists of three levels namely c1, c2 and c3. 
    Empty_Weight1 <- matrix(,nrow=tau.max, ncol=ncol(Phi)) 
    Empty_Weight2 <- matrix(,nrow=tau.max, ncol=ncol(Phi)) 
    Empty_Weight3 <- matrix(,nrow=tau.max, ncol=ncol(Phi)) 
    
    #genertaing random coefficients of weights for three class labels
    Empty_Weight1[1,] <- runif(ncol(Phi)) 
    Empty_Weight2[1,] <- runif(ncol(Phi)) 
    Empty_Weight3[1,] <- runif(ncol(Phi)) 

    #creating an error matrix
    error = matrix(,nrow=tau.max, ncol=1) 
    tau <- 1 #initializing tau value
    terminate <- FALSE #setting termination condition to false
    
    while(!terminate){
    
    train.index <- sample(1:train.len, replace = FALSE) #sampling the dataset
    Phi <- Phi[train.index,] #getting the basis function
    T <- T[train.index]  #getting train label indexes
    
    #For each point in the train data
    for (i in train.index){
        
    # Check the termination creteria
        if (tau == tau.max) {break}
        
        # Calculate the predictions of current models for current point
        score_weight1 = Empty_Weight1[tau,]%*%Phi[i,]
        score_weight2 = Empty_Weight2[tau,]%*%Phi[i,]
        score_weight3 = Empty_Weight3[tau,]%*%Phi[i,]
        max = max(score_weight1, score_weight2, score_weight3)

        
        # checking whether predictions match with the labels
        if (score_weight1 == max && T[i]!="C1"|score_weight2 == max && T[i]!="C2"|score_weight3 == max && T[i] !="C3"){
            
            tau = tau +1
            
            # Updating coefficients based on the conditions
            if (T[i]=='C1' && max == score_weight2){
                
                Empty_Weight1[tau,] = Empty_Weight1[tau-1,] + eta*Phi[i,]
                Empty_Weight2[tau,] = Empty_Weight2[tau-1,] - eta*Phi[i,]
                Empty_Weight3[tau,] = Empty_Weight3[tau-1,]
            
            }else if (T[i]=='C1' && max == score_weight3){
                
                Empty_Weight1[tau,] = Empty_Weight1[tau-1,] + eta*Phi[i,]
                Empty_Weight2[tau,] = Empty_Weight2[tau-1,]
                Empty_Weight3[tau,] = Empty_Weight3[tau-1,] - eta*Phi[i,]
            
            }else if (T[i]=='C2' && max == score_weight1){
                
                Empty_Weight1[tau,] = Empty_Weight1[tau-1,] - eta*Phi[i,]
                Empty_Weight2[tau,] = Empty_Weight2[tau-1,] + eta*Phi[i,]
                Empty_Weight3[tau,] = Empty_Weight3[tau-1,]
                
            }else if (T[i]=='C2' && max == score_weight3){
                
                Empty_Weight1[tau,] = Empty_Weight1[tau-1,]
                Empty_Weight2[tau,] = Empty_Weight2[tau-1,] + eta*Phi[i,]
                Empty_Weight3[tau,] = Empty_Weight3[tau-1,] - eta*Phi[i,]
            
            }else if (T[i]=='C3' && max == score_weight2){
                
                Empty_Weight1[tau,] = Empty_Weight1[tau-1,]
                Empty_Weight2[tau,] = Empty_Weight2[tau-1,] - eta*Phi[i,]
                Empty_Weight3[tau,] = Empty_Weight3[tau-1,] + eta*Phi[i,]
                
            }else if (T[i]=='C3' && max == score_weight1){
                
                Empty_Weight1[tau,] = Empty_Weight1[tau-1,] - eta*Phi[i,]
                Empty_Weight2[tau,] = Empty_Weight2[tau-1,]
                Empty_Weight3[tau,] = Empty_Weight3[tau-1,] + eta*Phi[i,]
               
            }
         
        }        
    }
    
     # Make predictions on entire test data using each Weighted vectors for three class labels
    predictions=as.data.frame(cbind(t(Empty_Weight1[tau,]%*%t(Phi)),t(Empty_Weight2[tau,]%*%t(Phi)),t(Empty_Weight3[tau,]%*%t(Phi))))
    
    for (row in 1:nrow(predictions)){
        
        predictions[row,'pred']=paste('C', which.max(predictions[row,]),sep = '')
        
    }
        
    #updating the error percentage
    error.percentage=1-(sum(predictions[,'pred']==T)/nrow(predictions))
    
    
    # recalculate termination conditions
    terminate <- (tau >= tau.max | (error.percentage<=epsilon))
}
   
    
    # Trim the dataframes
    Empty_Weight1 <- Empty_Weight1[1:tau,]
    Empty_Weight2 <- Empty_Weight2[1:tau,]
    Empty_Weight3 <- Empty_Weight3[1:tau,]
    
    return(list(Empty_Weight1, Empty_Weight2, Empty_Weight3))

}


### Train two multiclass perceptron models on the provided training data by setting the learning rates η to .09 and .01 respectively. 

In [49]:
set.seed(123)

#calling multiclass perceptron function for two diferent eta values
multiclassifier_coefficients1 <- multiclassfier_perceptron(train.data, train.label, eta = 0.09, epsilon = 0.001 , tau.max = 100 )
multiclassifier_coefficients2 <- multiclassfier_perceptron(train.data, train.label, eta = 0.01, epsilon = 0.001, tau.max = 100 )

#obtaining weight coefficients values for different class labels when eta = 0.09 
Empty_Weight1_eta1 = as.matrix(as.data.frame(multiclassifier_coefficients1[1]))
Empty_Weight2_eta1 = as.matrix(as.data.frame(multiclassifier_coefficients1[2]))
Empty_Weight3_eta1 = as.matrix(as.data.frame(multiclassifier_coefficients1[3]))

#obtaining weight coefficients values for different class labels when eta = 0.01
Empty_Weight1_eta2 = as.matrix(as.data.frame(multiclassifier_coefficients2[1]))
Empty_Weight2_eta2 = as.matrix(as.data.frame(multiclassifier_coefficients2[2]))
Empty_Weight3_eta2 = as.matrix(as.data.frame(multiclassifier_coefficients2[3]))

### Printing the weight coefficients values for different class labels with respect to eta values

In [50]:
#final weight coefficients of first empty matrix for eta = 0.09
tail(Empty_Weight1_eta1,1)

,X1,X2,X3,X4,X5
"[100,]",0.5575775,0.2723541,1.071988,0.1597626,0.1715491


In [51]:
#final weight coefficients of second empty matrix for eta = 0.09
tail(Empty_Weight2_eta1,1)

,X1,X2,X3,X4,X5
"[100,]",0.9455565,0.941313,0.4294689,0.2455358,0.2077444


In [52]:
#final weight coefficients of third empty matrix for eta = 0.09
tail(Empty_Weight3_eta1,1)

,X1,X2,X3,X4,X5
"[100,]",-0.2131667,0.5560776,0.47751,1.601787,1.120713


In [53]:
#final weight coefficients of first empty matrix for eta = 0.01
tail(Empty_Weight1_eta2, 1)

,X1,X2,X3,X4,X5
"[61,]",0.4457679,0.5446373,0.5605404,0.7247833,0.2805907


In [54]:
#final weight coefficients of second empty matrix for eta = 0.01
tail(Empty_Weight2_eta2, 1)

,X1,X2,X3,X4,X5
"[61,]",0.5233252,0.6430363,0.4967377,0.7902672,0.5372993


In [55]:
#final weight coefficients of third empty matrix for eta = 0.01
tail(Empty_Weight3_eta2, 1)

,X1,X2,X3,X4,X5
"[61,]",0.1762808,0.5953232,0.4218915,1.008623,0.8503176


### Using multi class prediction function to obtain the error percentage values for each batches

In [56]:
multi_class_prediction <- function(test.data, test.label, Empty_Weight1, Empty_Weight2, Empty_Weight3){
    
    #adding a new column to the train data which represents the coefficients namely w0
    test.Phi <- as.matrix(cbind(1, test.data))
    #creating an empty data frame with two columns
    error.percentage=data.frame(0, ncol=2)
    #giving column names to the created empty data frame
    colnames(error.percentage)=c('Batch', 'Error Percentage')
    
    #we will initialize batch number = 0 and then iterate through the batch-size of 5 
    batch.no = 0
    for (batch in seq(5,nrow(Empty_Weight1),5)){
        
        batch.no= batch.no + 1
        
        # Make predictions on entire test data using each Weighted vectors for three class labels
        predictions=as.data.frame(cbind(t(Empty_Weight1[batch,]%*%t(test.Phi)),t(Empty_Weight2[batch,]%*%t(test.Phi)),t(Empty_Weight3[batch,]%*%t(test.Phi))))     
        
        for (row in 1:nrow(predictions)){
            
            predictions[row,'pred']=paste('C', which.max(predictions[row,]),sep = '')
            
        }
         
        error.percentage[batch.no,1]=batch.no
        error.percentage[batch.no,2]=1-(sum(predictions[,'pred']==test.label)/nrow(predictions))

}
    
    return(list(predictions,error.percentage))
    
}

In [57]:
#calling the multi class prediction function for two different eta values
error_predictions1 = multi_class_prediction(test.data, test.label, Empty_Weight1_eta1, Empty_Weight2_eta1, Empty_Weight3_eta1)
error_predictions2 = multi_class_prediction(test.data, test.label, Empty_Weight1_eta2, Empty_Weight2_eta2, Empty_Weight3_eta2)

In [58]:
head(error_predictions1)

V1,V2,V3,pred
<dbl>,<dbl>,<dbl>,<chr>
-0.308120627,-0.748794527,-4.5037841,C1
-0.150456674,-0.967282375,-4.4990838,C1
2.122130977,0.780110599,-2.6227877,C1
0.709356380,-0.233522198,-3.9127098,C1
-0.078166891,-0.665202037,-4.4559665,C1
0.654735215,-0.439733912,-3.9517265,C1
-0.549733853,-1.479340288,-5.3190939,C1
3.420203295,1.571489687,-2.0660414,C1
0.999526180,-0.011414709,-3.6779977,C1


In [59]:
head(error_predictions2)

V1,V2,V3,pred
<dbl>,<dbl>,<dbl>,<chr>
-1.5587919,-2.0024716,-3.0116628,C1
-1.5810180,-2.0708051,-3.0663376,C1
0.1082959,-0.3376407,-1.3863919,C1
-0.9432672,-1.4330979,-2.4971228,C1
-1.4279867,-1.9173275,-2.9735191,C1
-1.0294577,-1.5387336,-2.5788825,C1
-2.1025976,-2.6588020,-3.7191936,C1
0.8552751,0.3605994,-0.8086135,C1
-0.7550617,-1.2182945,-2.2719710,C1


In [60]:
#melt the data frame based on the batch id
predictions1 = error_predictions1[1]
error.percentage1 = error_predictions1[2]
error.m1 = melt(error.percentage1, id='Batch')

#melt the data frame based on the batch id
predictions2 = error_predictions2[1]
error.percentage2 = error_predictions2[2]
error.m2 = melt(error.percentage2, id='Batch')

In [65]:
#plot the graph for batch size versus error rate
options(repr.plot.width = 10, repr.plot.height = 8)
plot(error.m1$Batch, error.m1$value, type = "l", col = "red", xlab = "Batch size", ylab = "error", lwd = 5)
par(new=TRUE)
plot(error.m2$Batch, error.m2$value, type = "l", col = "blue",  xlab = "Batch size", ylab = "error", lwd = 5)

title("Plotting a graph for two different eta values for batch size versus error percentages")

legend(x="topright", legend=c("eta = 0.09","eta = 0.01"), fill=c("red","blue"))

### Now, explain how the testing errors of two models behave differently, as the training data increases, by observing your plot

In the above graph, per each batch , the test data takes five training weighted vectors obtained from multiclass_perceptron model. Initially, the weighted vectors of the three classes are taken randomly using 'runif' function in R. But, when we start updating the weights of weighted vectors for each class in each iteration, to get the good possible decision boundary line that separates the three classes, the number of miss-classifications decreases. Thereby, as we increase the batch size, we are getting the optimal weighted values of each class, which thereby decreases the missclassifications in test data set and thereby errors in the data set decreases.

Therefore, we can conclude that with increase of batch size, the number of miss-classifications decreases.

When comes to comparing of eta values, eta stands for learning rate. Lower learning rate always have a lower error value. Because, if the learning rate is more, then it bounces farward and backward and sometimes may end up in an infinite loop, without properly classifying all the training data points. 

Therefore, from the given eta values, 0.01 is having the less error rate with increase of batch size.

## References:-

https://www.kaggle.com/anlgrbz/implementation-of-multiclass-perceptron